In [50]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
import time
import os

classifiers = {
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Naive Bayes": GaussianNB(),
    "Support Vector Machine": SVC(probability=True),
    "XGBoost": XGBClassifier(),
    "Decision Tree": DecisionTreeClassifier()
}

k_folds = [10, 20, 30]

results_df = pd.DataFrame(columns=["Dataset", "K-Fold", "Classifier", "Accuracy", "AUC", "Training Time (s)", "Testing Time (s)"])

for dataset_number in range(1, 8):
    dataset_filename = f"E:/R(5)/Dataset_{dataset_number}.csv"
    if not os.path.exists(dataset_filename):
        continue

    data = pd.read_csv(dataset_filename)
    X = data.drop('label', axis=1)
    y = data['label']

    for k in k_folds:
        for clf_name, clf in classifiers.items():
            kf = KFold(n_splits=k, shuffle=True, random_state=42)
            
            accuracy_scores = []
            auc_scores = []
            train_times = []
            test_times = []

            for train_index, test_index in kf.split(X):
                X_train, X_test = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]

                start_time = time.time()
                clf.fit(X_train, y_train)
                train_time = time.time() - start_time

                start_time = time.time()
                y_pred = clf.predict(X_test)
                test_time = time.time() - start_time

                accuracy = accuracy_score(y_test, y_pred)
                accuracy_scores.append(accuracy)

                try:
                    probas = clf.predict_proba(X_test)
                    auc_value = roc_auc_score(y_test, probas[:, 1])
                    auc_scores.append(auc_value)
                except AttributeError:
                    auc_scores.append(None)

                train_times.append(train_time)
                test_times.append(test_time)

            mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
            mean_auc = sum(auc_scores) / len(auc_scores)
            mean_train_time = sum(train_times) / len(train_times)
            mean_test_time = sum(test_times) / len(test_times)

            results_df = results_df.append({
                "Dataset": f"Dataset_{dataset_number}",
                "K-Fold": k,
                "Classifier": clf_name,
                "Accuracy": f"{mean_accuracy:.2f}",
                "AUC": f"{mean_auc:.2f}",
                "Training Time (s)": f"{mean_train_time:.2f}",
                "Testing Time (s)": f"{mean_test_time:.2f}"
            }, ignore_index=True)

pivot_df1 = results_df.pivot_table(index=["Dataset", "K-Fold"], columns="Classifier", values=["Accuracy"])
pivot_df2 = results_df.pivot_table(index=["Dataset", "K-Fold"], columns="Classifier", values=["AUC"])
pivot_df3 = results_df.pivot_table(index=["Dataset", "K-Fold"], columns="Classifier", values=["Training Time (s)", "Testing Time (s)"])

C:\Users\acer\AppData\Local\Temp\ipykernel_9108\1861087510.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\acer\AppData\Local\Temp\ipykernel_9108\1861087510.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\acer\AppData\Local\Temp\ipykernel_9108\1861087510.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\acer\AppData\Local\Temp\ipykernel_9108\1861087510.py:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\acer\AppData\Local\Temp\ipykernel_9108\1861087510.py:90

In [57]:
pivot_df1

Accuracy                                              \
Classifier       AdaBoost Decision Tree Gradient Boosting Naive Bayes   
Dataset   K-Fold                                                        
Dataset_1 10         1.00          0.98              0.99        0.94   
          20         0.99          0.98              0.99        0.94   
          30         1.00          0.98              1.00        0.94   
Dataset_2 10         0.97          0.95              0.96        0.93   
          20         0.97          0.96              0.97        0.93   
          30         0.97          0.96              0.96        0.92   
Dataset_3 10         0.92          0.92              0.92        0.91   
          20         0.92          0.92              0.92        0.91   
          30         0.92          0.92              0.92        0.91   
Dataset_4 10         1.00          0.97              0.98        0.95   
          20         1.00          0.98              0.98        0.95   
          30         1.00          0.97              0.98        0.95   
Dataset_5 10         0.99          0.98              0.99        0.95   
          20         0.99          0.98              0.99        0.95   
          30         0.99          0.98              1.00        0.95   
Dataset_6 10         0.96          0.95              0.98        0.93   
          20         0.96          0.96              0.98        0.93   
          30         0.96          0.95              0.98        0.92   
Dataset_7 10         0.97          0.95              0.95        0.94   
          20         0.97          0.95              0.95        0.94   
          30         0.98          0.95              0.96        0.94   

                                                               
Classifier       Random Forest Support Vector Machine XGBoost  
Dataset   K-Fold                                               
Dataset_1 10              1.00                   0.95    0.99  
          20              1.00                   0.95    0.99  
          30              1.00                   0.95    1.00  
Dataset_2 10              0.97                   0.95    0.97  
          20              0.97                   0.95    0.97  
          30              0.97                   0.96    0.97  
Dataset_3 10              0.92                   0.92    0.91  
          20              0.92                   0.92    0.91  
          30              0.92                   0.92    0.92  
Dataset_4 10              0.99                   0.97    0.99  
          20              0.99                   0.98    0.99  
          30              0.99                   0.98    1.00  
Dataset_5 10              0.99                   0.98    0.99  
          20              1.00                   0.99    0.99  
          30              1.00                   0.99    1.00  
Dataset_6 10              0.98                   0.93    0.98  
          20              0.97                   0.93    0.98  
          30              0.98                   0.93    0.98  
Dataset_7 10              0.97                   0.94    0.95  
          20              0.96                   0.94    0.96  
          30              0.97                   0.95    0.96

In [59]:
pivot_df2

AUC                                              \
Classifier       AdaBoost Decision Tree Gradient Boosting Naive Bayes   
Dataset   K-Fold                                                        
Dataset_1 10         1.00          0.98              1.00        0.99   
          20         1.00          0.98              1.00        0.99   
          30         1.00          0.98              1.00        0.98   
Dataset_2 10         0.99          0.98              0.99        0.98   
          20         0.99          0.99              0.99        0.98   
          30         0.99          0.98              0.99        0.98   
Dataset_3 10         0.94          0.94              0.94        0.93   
          20         0.94          0.94              0.94        0.93   
          30         0.94          0.94              0.94        0.93   
Dataset_4 10         1.00          0.98              1.00        0.99   
          20         1.00          0.98              1.00        0.99   
          30         1.00          0.98              1.00        0.99   
Dataset_5 10         1.00          0.98              1.00        0.99   
          20         1.00          0.98              1.00        0.99   
          30         1.00          0.98              1.00        0.99   
Dataset_6 10         0.99          0.95              1.00        0.98   
          20         0.99          0.95              1.00        0.98   
          30         1.00          0.95              1.00        0.98   
Dataset_7 10         1.00          0.95              0.99        0.98   
          20         1.00          0.95              0.99        0.98   
          30         0.99          0.96              0.99        0.97   

                                                               
Classifier       Random Forest Support Vector Machine XGBoost  
Dataset   K-Fold                                               
Dataset_1 10              1.00                   1.00    1.00  
          20              1.00                   1.00    1.00  
          30              1.00                   1.00    1.00  
Dataset_2 10              0.99                   0.99    0.99  
          20              0.99                   0.99    0.99  
          30              0.99                   0.99    0.99  
Dataset_3 10              0.94                   0.94    0.94  
          20              0.94                   0.94    0.94  
          30              0.94                   0.94    0.94  
Dataset_4 10              1.00                   1.00    1.00  
          20              1.00                   1.00    1.00  
          30              1.00                   1.00    1.00  
Dataset_5 10              1.00                   1.00    1.00  
          20              1.00                   1.00    1.00  
          30              1.00                   1.00    1.00  
Dataset_6 10              1.00                   0.98    1.00  
          20              1.00                   0.98    1.00  
          30              1.00                   0.98    1.00  
Dataset_7 10              1.00                   0.99    0.99  
          20              0.99                   0.99    0.99  
          30              1.00                   0.99    0.99

In [60]:
pivot_df3.to_csv('E:/R(5)/r_3.csv')